In [1]:
#!pip install opencv-python keras tensorflow numpy matplotlib tqdm pandas scikit-learn gradio transformers



In [2]:
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

In [1]:
import torch
print(torch.__version__)  # Should print version number
print(torch.cuda.is_available())  # Should print True


2.6.0+cu118
True


In [2]:
import tensorflow as tf

# Force TensorFlow to use GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ TensorFlow is using GPU!")
    except RuntimeError as e:
        print(e)


✅ TensorFlow is using GPU!


In [5]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Dataset Path
DATASET_PATH = "UTKFace/"
IMG_SIZE = 64  

# Lists to store data
ages, genders, images = [], [], []

# Read images and labels
for img_name in tqdm(os.listdir(DATASET_PATH)):
    if img_name.endswith(".jpg"):
        try:
            # Parse filename (format: age_gender_race.jpg)
            age, gender, _ = img_name.split("_")[:3]
            age, gender = int(age), int(gender)  # 0 = Male, 1 = Female

            # Read and preprocess image
            img = cv2.imread(os.path.join(DATASET_PATH, img_name))
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0  # Normalize

            # Store data
            images.append(img)
            ages.append(age)
            genders.append(gender)
        except Exception as e:
            continue  # Skip corrupted files

# Convert to numpy arrays
X = np.array(images)
y_age = np.array(ages)
y_gender = to_categorical(np.array(genders), 2)  # Male=0, Female=1

# Split into train & test sets
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(
    X, y_age, y_gender, test_size=0.2, random_state=42
)

print("✅ Data Loaded: ", X_train.shape)


100%|███████████████████████████████████████████████████████████████████████████| 23708/23708 [00:35<00:00, 672.13it/s]


✅ Data Loaded:  (18966, 64, 64, 3)


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# CNN Model for Gender Prediction (Classification)
gender_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 Classes: Male & Female
])

# Compile & Train Model
gender_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
gender_model.fit(X_train, y_gender_train, epochs=30, batch_size=32, validation_data=(X_test, y_gender_test))

# Save Model
gender_model.save("gender_model.h5")
print("✅ Gender Model Saved!")


Epoch 1/30
593/593 [==============================] - 10s 13ms/step - loss: 0.4489 - accuracy: 0.7895 - val_loss: 0.3368 - val_accuracy: 0.8555
Epoch 2/30
593/593 [==============================] - 7s 12ms/step - loss: 0.3187 - accuracy: 0.8601 - val_loss: 0.3014 - val_accuracy: 0.8667
Epoch 3/30
593/593 [==============================] - 8s 13ms/step - loss: 0.2887 - accuracy: 0.8745 - val_loss: 0.2740 - val_accuracy: 0.8842
Epoch 4/30
593/593 [==============================] - 7s 11ms/step - loss: 0.2629 - accuracy: 0.8862 - val_loss: 0.2617 - val_accuracy: 0.8865
Epoch 5/30
593/593 [==============================] - 7s 12ms/step - loss: 0.2459 - accuracy: 0.8934 - val_loss: 0.2641 - val_accuracy: 0.8889
Epoch 6/30
593/593 [==============================] - 8s 13ms/step - loss: 0.2295 - accuracy: 0.9028 - val_loss: 0.2570 - val_accuracy: 0.8872
Epoch 7/30
593/593 [==============================] - 8s 13ms/step - loss: 0.2183 - accuracy: 0.9064 - val_loss: 0.2612 - val_accuracy: 0.889

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# CNN Model for Age Prediction (Regression)
age_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Regression output
])

# Compile & Train Model
age_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
age_model.fit(X_train, y_age_train, epochs=40, batch_size=32, validation_data=(X_test, y_age_test),verbose=1)

# Save Model
age_model.save("age_model.h5")
print("✅ Age Model Saved!")


Epoch 1/40
593/593 [==============================] - 9s 13ms/step - loss: 398.9722 - mae: 15.3243 - val_loss: 232.6627 - val_mae: 11.4557
Epoch 2/40
593/593 [==============================] - 7s 12ms/step - loss: 239.7574 - mae: 11.7469 - val_loss: 177.3626 - val_mae: 9.8171
Epoch 3/40
593/593 [==============================] - 8s 13ms/step - loss: 193.7592 - mae: 10.4770 - val_loss: 156.3341 - val_mae: 9.0719
Epoch 4/40
593/593 [==============================] - 7s 13ms/step - loss: 166.7695 - mae: 9.6265 - val_loss: 128.9050 - val_mae: 8.4908
Epoch 5/40
593/593 [==============================] - 7s 12ms/step - loss: 151.5294 - mae: 9.1201 - val_loss: 119.5652 - val_mae: 7.8994
Epoch 6/40
593/593 [==============================] - 7s 12ms/step - loss: 136.9631 - mae: 8.6476 - val_loss: 109.5841 - val_mae: 7.6410
Epoch 7/40
593/593 [==============================] - 8s 14ms/step - loss: 131.5193 - mae: 8.4753 - val_loss: 107.3355 - val_mae: 7.5869
Epoch 8/40
593/593 [=================

In [31]:
import torch
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# ✅ Set Device to GPU (if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using Device:", device)

# ✅ Load BLIP Model & Processor from Local Folder
processor = BlipProcessor.from_pretrained(r"C:\Users\OMEN\Desktop\python\cmp\Hugging\blip")
model = BlipForConditionalGeneration.from_pretrained(r"C:\Users\OMEN\Desktop\python\cmp\Hugging\blip").to(device)

# ✅ Load Age & Gender Detection Models
age_model = load_model("age_model.h5")      # Ensure these files exist
gender_model = load_model("gender_model.h5")


Using Device: cuda


In [ ]:
#!pip install torch transformers timm fairscale


In [5]:
from tensorflow.keras.models import load_model

# Load Trained Models
age_model = load_model("age_model.h5")
gender_model = load_model("gender_model.h5")

print("✅ Models loaded successfully!")


✅ Models loaded successfully!


In [6]:
from torchvision import transforms  # <-- Add this line
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# Load BLIP from your local path
blip_path = r"C:\Users\OMEN\Desktop\python\cmp\Hugging\blip"

device = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained(blip_path)
model = BlipForConditionalGeneration.from_pretrained(blip_path).to(device)


from PIL import Image

def generate_description(image, predicted_age, predicted_gender):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Convert NumPy array to PIL Image if necessary
    if isinstance(image, np.ndarray):
        image = Image.fromarray(image)

    # Transform (Resize + Convert to Tensor)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize image to 224x224 for BLIP
        transforms.ToTensor()  # Convert to tensor (No normalization)
    ])
    
    image_tensor = transform(image).unsqueeze(0).to(device)

    # Convert back to PIL Image for BLIP
    image_pil = transforms.ToPILImage()(image_tensor.squeeze(0))

    # Ensure gender is a valid string
    predicted_gender = "Male" if predicted_gender == 0 else "Female"

    # Generate description using BLIP
    with torch.no_grad():
        inputs = processor(images=image_pil, return_tensors="pt").to(device)
        output = model.generate(**inputs)

    description = processor.batch_decode(output, skip_special_tokens=True)[0]

    # Final formatted description
    final_description = f"{description} They appear to be a {predicted_gender} around {predicted_age} years old."
    
    return final_description


In [7]:
import gradio as gr
import tensorflow as tf
import cv2
import numpy as np
import torch

# Ensure TensorFlow uses GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Load Trained Age & Gender Models on GPU
with tf.device("/GPU:0"):
    age_model = tf.keras.models.load_model("age_model.h5")
    gender_model = tf.keras.models.load_model("gender_model.h5")

# Function to Predict & Generate Descriptions
def predict_and_describe(image):
    if isinstance(image, dict):  # Ensure image is in NumPy format
        image = image["image"]

    image = np.asarray(image, dtype=np.uint8)  # Convert to NumPy array
    image_resized = cv2.resize(image, (64, 64)) / 255.0
    image_resized = np.expand_dims(image_resized, axis=0)

    # Predict Age & Gender on GPU
    with tf.device("/GPU:0"):
        predicted_age = int(age_model.predict(image_resized)[0][0])
        predicted_gender = np.argmax(gender_model.predict(image_resized))

    # Generate Description using BLIP on GPU
    with torch.autocast("cuda"):  
        description = generate_description(image, predicted_age, predicted_gender)

    return f"👤 Age: {predicted_age} | Gender: {'Male' if predicted_gender == 0 else 'Female'}\n📜 Description: {description}"

# ✅ Use `gr.Interface()` Instead of `gr.Blocks()`
interface = gr.Interface(
    fn=predict_and_describe,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Textbox(label="Result"),
    title="Age & Gender Detection with BLIP Description",
    description="Upload an image to detect age, gender, and generate a description."
)

# Launch the Gradio app
interface.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
